# TRURL 2 on Amazon SageMaker

## Setup

In [ ]:
import json
import logging

import boto3
import sagemaker

from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

# Reducing verbosity of the `sagemaker` library.

sagemaker_config_logger = logging.getLogger('sagemaker.config')
sagemaker_logger = logging.getLogger('sagemaker')

sagemaker_config_logger.setLevel(logging.ERROR)
sagemaker_logger.setLevel(logging.ERROR)

In [ ]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f'Amazon SageMaker IAM Role ARN: {role}')

## Deploying model into Amazon SageMaker

In [ ]:
model_name = 'trurl-2-7b'

instance_type = 'ml.g5.2xlarge'
num_of_gpus = 1

container_startup_timeout = 300

In [ ]:
env = {
    'HF_MODEL_ID': f'Voicelab/{model_name}',
    'SM_NUM_GPUS': json.dumps(num_of_gpus)
}

hf_image_uri = get_huggingface_llm_image_uri('huggingface', version='1.1.0')

huggingface_model = HuggingFaceModel(
	image_uri=hf_image_uri,
	env=env,
	role=role, 
)

In [ ]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=container_startup_timeout,
    endpoint_name='example-trurl2-endpoint'
)

## Let's test the freshly deployed endpoint!

In [ ]:
data = {
    'inputs': '<s>[INST]Kim jest Stanisław Lem?[/INST]',
    'parameters': {
        'do_sample': True,
        'top_p': 0.6,
        'temperature': 0.9,
        'top_k': 50,
        'max_new_tokens': 100,
        'repetition_penalty': 1.05,
        'stop': ['</s>']
    }
}

predictor.predict(data)

In [ ]:
data = {
    'inputs': '<s>[INST]Czym zajmuje się firma VoiceLab?[/INST]',
    'parameters': {
        'do_sample': True,
        'top_p': 0.6,
        'temperature': 0.9,
        'top_k': 50,
        'max_new_tokens': 100,
        'repetition_penalty': 1.05,
        'stop': ['</s>']
    }
}

predictor.predict(data)

## Okay, it's time for something more complicated!

In [ ]:
!pip install mediawikiapi==1.2.0

In [ ]:
import pandas as pd

from mediawikiapi import MediaWikiAPI

mediawikiapi = MediaWikiAPI()
mediawikiapi.config.language = 'pl'

pl_nobel_prizes_page = mediawikiapi.page('Lista_laureatów_Nagrody_Nobla_związanych_z_Polską')
pl_nobel_prizes_df = pd.read_html(pl_nobel_prizes_page.url, attrs={'class': 'wikitable'})[0]
pl_nobel_prizes = pl_nobel_prizes_df.drop([0, 1, 4, 5], axis=1).drop([0], axis=0).to_string(index=False, header=False)

print(pl_nobel_prizes)

In [ ]:
prompt = f"""<s>[INST] 
<<SYS>> 
Na podstawie przygotowanego kontekstu odpowiedz na zadane pytanie. Jeśli nie ma tej informacji w kontekście, nie dodawaj niczego. Jeśli nie znasz odpowiedzi, nie odpowiadaj. Odpowiadaj jak najbardziej zwięźle.
<</SYS>> 
Tabela z danymi Polek i Polaków którzy otrzymali nagrody Nobla: 

{pl_nobel_prizes} 

Jako pierwsze słowo w pierwszej kolumnie mamy imię, potem nazwisko, oraz w nawiasach rok urodzenia oraz śmierci. Załóż, że polskie kobiece imiona kończą się na literę a. W drugiej kolumnie mamy kategorię przyznanej nagrody Nobla. Jeśli dana osoba otrzymała więcej niż jedną nagrodę, w tej kolumnie jest więcej niż jedno słowo. Dodatkowo, w takim przypadku przy każdej kategorii w nawiasie jest rok przynania nagrody.

Podaj sumaryczną liczbę nagród Nobla które przynano polskim kobietom.
[/INST]
"""

data = {
    'inputs': prompt,
    'parameters': {
        'do_sample': True,
        'top_p': 0.6,
        'temperature': 0.9,
        'top_k': 50,
        'max_new_tokens': 50,
        'repetition_penalty': 1.05,
        'stop': ['</s>']
    }
}

response = predictor.predict(data)
text = response[0]['generated_text']
print(text[len(prompt):])

## Let's make it conversational!

In [ ]:
prompt = """<s>[INST]
<<SYS>> 
Jesteś pomocnym asystentem. Będziemy grać w grę o nazwie '20 pytań'. Twoim zadaniem jest odgadnąć co mam na myśli, zadając po jednym pytaniu na które mogę odpowiedzie tylko 'tak' lub 'nie'. Gra dobiegnie końca gdy wyczerpie się limit 20 pytań, lub odgadniesz konkretną rzecz wcześniej.
<</SYS>> 
Jestem gotów, zadaj pierwsze pytanie.
[/INST]
"""

data = {
    'inputs': prompt,
    'parameters': {
        'do_sample': True,
        'top_p': 0.6,
        'temperature': 0.9,
        'top_k': 50,
        'max_new_tokens': 512,
        'repetition_penalty': 1.05,
        'stop': ['</s>']
    }
}

response = predictor.predict(data)
text = response[0]['generated_text']
print(text[len(prompt):])

## Enough exploration - it's time to clean-up and develop something!

In [ ]:
predictor.delete_model()

In [ ]:
predictor.delete_endpoint()